In [ ]:
import polars as pl
import networkx as nx
import random
import rtsvg
rt = rtsvg.RACETrack()

# Create a graph
from linknode_graph_patterns import LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()
g          = _patterns_.createPattern('mesh')
#g          = nx.Graph()
#g.add_edge('a', 'b', weight=1.0), g.add_edge('a', 'c', weight=1.0), g.add_edge('b', 'c', weight=1.0), g.add_edge('c','d', weight=1.0), g.add_edge('d','e', weight=1.0)

# Create the distance dataframe from the graph
_lu_    = {'fm':[], 'to':[], 't':[], 's':[]}
for node in g.nodes:
    for nbor in g.neighbors(node):
        _w_ = g[node][nbor]['weight'] if 'weight' in g[node][nbor] else 1.0
        _lu_['fm'].append(node), _lu_['to'].append(nbor), _lu_['t'].append(_w_), _lu_['s'].append(False)
df_dist = pl.DataFrame(_lu_)

# Create positional dataframe w/ random values
_lu_    = {'node':[], 'x':[], 'y':[]}
for node in g.nodes: _lu_['node'].append(node), _lu_['x'].append(random.random()), _lu_['y'].append(random.random())
df_pos  = pl.DataFrame(_lu_)

# Run the spring simulation
positions = []
k, mu     = 2, 1.0/(2.0*len(g.nodes))
__dx__, __dy__ = (pl.col('x') - pl.col('x_right')), (pl.col('y') - pl.col('y_right'))
for i in range(2*len(g.nodes)):
  positions.append(df_pos)
  df_pos = df_pos.join(df_pos, how='cross') \
                 .filter(pl.col('node') != pl.col('node_right')) \
                 .join(df_dist, left_on=['node', 'node_right'], right_on=['fm','to']) \
                 .with_columns((__dx__**2 + __dy__**2).sqrt().alias('d')) \
                 .with_columns(pl.col('t').pow(k).alias('t_k')) \
                 .with_columns(pl.when(pl.col('d') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('d')).alias('d'),
                               pl.when(pl.col('t') < 0.001).then(pl.lit(0.001)).otherwise(pl.col('t')).alias('t')) \
                 .with_columns((pl.col('t')**(2-k)).alias('__prod_1__'),
                               ((2.0*__dx__*(1.0 - pl.col('t')/pl.col('d')))/pl.col('t_k')).alias('xadd'),
                               ((2.0*__dy__*(1.0 - pl.col('t')/pl.col('d')))/pl.col('t_k')).alias('yadd'),
                               (((pl.col('t') - pl.col('d'))**2)/(pl.col('t_k'))**k).alias('__prod_2__')) \
                 .group_by(['node','x','y','s']).agg(pl.col('xadd').sum(), pl.col('yadd').sum(), pl.col('__prod_1__').sum(), pl.col('__prod_2__').sum()) \
                 .with_columns(pl.when(pl.col('s')).then(pl.col('x')).otherwise(pl.col('x') - mu * pl.col('xadd')).alias('x'),
                               pl.when(pl.col('s')).then(pl.col('y')).otherwise(pl.col('y') - mu * pl.col('yadd')).alias('y')) \
                 .drop(['xadd','yadd', 's', '__prod_1__', '__prod_2__'])
positions.append(df_pos)

# Make an animation
_dur_ = '5s'
x0, y0, x1, y1 = positions[0]['x'].min(), positions[0]['y'].min(), positions[0]['x'].max(), positions[0]['y'].max()
for _pos_ in positions: x0, y0, x1, y1 = min(x0, _pos_['x'].min()), min(y0, _pos_['y'].min()), max(x1, _pos_['x'].max()), max(y1, _pos_['y'].max())
svg = [f'<svg x="0" y="0" width="256" height="256" viewBox="{x0} {y0} {x1-x0} {y1-y0}">']
svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="white" />')
node_positions = {}
for _node_ in g.nodes: node_positions[_node_] = []
for _pos_ in positions:
    for i in range(len(_pos_)):
        _node_, _x_, _y_ = _pos_['node'][i], _pos_['x'][i], _pos_['y'][i]
        node_positions[_node_].append((_x_, _y_))
for _node_ in g.nodes:
    for _nbor_ in g.neighbors(_node_):
        _x1_, _y1_ = node_positions[_node_][0]
        _x2_, _y2_ = node_positions[_nbor_][0]
        x1_strs, y1_strs = [], []
        for _x_, _y_ in node_positions[_node_]: x1_strs.append(f'{_x_}'), y1_strs.append(f'{_y_}')
        x2_strs, y2_strs = [], []
        for _x_, _y_ in node_positions[_nbor_]: x2_strs.append(f'{_x_}'), y2_strs.append(f'{_y_}')
        svg.append(f'<line x1="{_x1_}" y1="{_y1_}" x2="{_x2_}" y2="{_y2_}" stroke="black" stroke-width="0.002">')
        svg.append(f'<animate attributeName="x1" values="{";".join(x1_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="y1" values="{";".join(y1_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="x2" values="{";".join(x2_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append(f'<animate attributeName="y2" values="{";".join(y2_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
        svg.append('</line>')

for _node_ in node_positions:
    _x_, _y_ = node_positions[_node_][0]
    svg.append(f'<circle cx="{_x_}" cy="{_y_}" r="0.01" fill="black">')
    x_strs, y_strs = [], []
    for _x_, _y_ in node_positions[_node_]: x_strs.append(f'{_x_}'), y_strs.append(f'{_y_}')
    svg.append(f'<animate attributeName="cx" values="{";".join(x_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="cy" values="{";".join(y_strs)}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</circle>')
svg.append('</svg>')
rt.tile([''.join(svg)])